In [4]:
from functools import reduce
import statistics as st
import cv2
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
#import matplotlib.pyplot as plt
import os
#from PyPDF2 import PdfReader
#%matplotlib inline
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\epenago\AppData\Local\Tesseract-OCR\tesseract.exe'

In [5]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

In [142]:
def calcular_espacios(diccionario, indices):
    
    suma = 0
    bandera = True
  
    width_espacios = []
    for i in range(len(indices)):
             
        if(bandera == True):
            
            bandera = False
            suma = 0
            suma += int(diccionario["left"][indices[i]])+int(diccionario["width"][indices[i]])

            if i < len(indices)-1:
                if int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+10 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-10:
                    pass
                else:
                    bandera = True
            else:           
                    bandera = True
                    

        elif(i < len(indices)-1):
            width_espacios.append(diccionario["left"][indices[i]] - suma)
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            if int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+10 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-10:
                pass
            else:
                bandera = True
               

        elif(i <= len(indices)-1):
            width_espacios.append(diccionario["left"][indices[i]] - suma)
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            bandera = True
            
    #print(width_espacios)
    width_espacios.sort() 
    
    moda = st.mode(width_espacios)
    return (moda-(moda*2), moda+(moda*2))

In [7]:
def coordenadas_extraccion(diccionario, indices, intervalo):
    
    datos_extraidos = []
    dict = {}
    coordenadas = []
    suma = 0
    bandera = True
    indice_anterior = -1
    strings = []
    string = ""
    index = []
    line = 0

    for i in range(len(indices)):
             
        if(bandera == True):
            indice_anterior = i
            
            bandera = False
            suma = 0
            suma += int(diccionario["left"][indices[i]])+int(diccionario["width"][indices[i]])

            # Se guarda la palabra y su indice
            string = diccionario["text"][indices[i]]
            index.append(indices[i])
            
            if i < len(indices)-1:
                if (int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+13 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-13):
                    
                    if (diccionario["left"][indices[i+1]] - suma) < intervalo[0] or (diccionario["left"][indices[i+1]] - suma) > intervalo[1]:
                        bandera = True
                        #print(indices[i])
                        coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                        diccionario["top"][indices[indice_anterior]], 
                        suma,
                        diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])
                        
                        dict["text"] = string
                        dict["line"] = line
                        dict["index"] = index
                        dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                        datos_extraidos.append(dict)
                                                                    
                        dict = {}                        
                        string = ""
                        index = []

                        bandera = True
                        indice_anterior = -1
                else:
                    bandera = True
                    #print(indices[i])
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])
                    

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []
                    line += 1

                    bandera = True
                    indice_anterior = -1
            else:           
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []

                    bandera = True
                    indice_anterior = -1

        elif(i < len(indices)-1):
            string += (" " + diccionario["text"][indices[i]])
            index.append(indices[i])

            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            if (int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+13 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-13):
                
                if (diccionario["left"][indices[i+1]] - suma) < intervalo[0] or (diccionario["left"][indices[i+1]] - suma) > intervalo[1]:
                    bandera = True
                    #print(indices[i])
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []

                    bandera = True
                    indice_anterior = -1
            else:
                bandera = True
                #print(indices[i])
                coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                diccionario["top"][indices[indice_anterior]], 
                suma,
                diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                dict["text"] = string
                dict["line"] = line
                dict["index"] = index
                dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                datos_extraidos.append(dict)
                                                            
                dict = {}                        
                string = ""
                index = []
                line += 1

                bandera = True
                indice_anterior = -1

        elif(i <= len(indices)-1):
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            coordenadas.append([diccionario["left"][indices[indice_anterior]], 
            diccionario["top"][indices[indice_anterior]], 
            suma,
            diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

            string = ""
            index = []

            bandera = True
            indice_anterior = -1

    return (coordenadas, datos_extraidos)

En el Metodo extraccion_columns logramos separar las columnas extrayedo y clasificando los indices en el dicionario data_ordenaizer

In [82]:
def extracion_colums(df):

    arr = np.array(df["coordinates"].to_list())
    colums = []
    
    for i in np.arange(len(arr)):
        temp = []
        temp.append(i)
        for j in np.arange(len(arr)):

            if i == j: continue
            interval1 = np.arange(arr[i,0], arr[i,1]+1, dtype=int)
            interval2 = np.arange(arr[j,0], arr[j,1]+1, dtype=int)

            sum = 0
            for k in interval1:
                if k in interval2:
                    sum += 1
    
            tamaño1 = len(interval1)
            tamaño2 = len(interval2)
            percent1 = int((100/tamaño1)*sum )
            percent2 = int((100/tamaño2)*sum )
            """
            print("tamaño1: ", tamaño1)
            print("tamaño2: ", tamaño2)
            print("la suma: ", sum)
            print("percent1: ", percent1, "%")
            print("percent2: ", percent2, "%")
            """
            if percent1 > 55 and percent2 > 55:
                temp.append(j)
        
        colums.append(temp)
        
    cl = []
    descartados = []
    for i in np.arange(len(colums)):
        temp = []
        temp = colums[i]
        if i in descartados: continue
        for j in np.arange(len(colums)):

            if i == j: continue
            interval1 = colums[i]
            interval2 = colums[j]

            sum = 0
            for k in interval1:
                if k in interval2:
                    sum += 1
    
            tamaño1 = len(interval1)
            tamaño2 = len(interval2)
            percent1 = int((100/tamaño1)*sum )
            percent2 = int((100/tamaño2)*sum )
            """
            print("tamaño1: ", tamaño1)
            print("tamaño2: ", tamaño2)
            print("la suma: ", sum)
            print("percent1: ", percent1, "%")
            print("percent2: ", percent2, "%")
            """
            if percent1 > 20 or percent2 > 20:
                temp += colums[j]
                descartados.append(j)
        cl.append(temp)

    colums = []    
    #se eliminan elementos repetidos en las columnas
    for element in cl:
        temp = []
        for i in element:
            if i not in temp:
                temp.append(i)
        colums.append(temp)
    

    print("el tamaño de cl es: ", len(cl))
    #en este bloque se saca la pocision en pixeles de la imagen
    min_max = []
    for element in colums:   
        menor = arr[element[0],0]
        mayor = arr[element[0],1]
        for i in element:
            if arr[i,0] < menor:
                menor = arr[i,0]
            
            if arr[i,1] > mayor:
                mayor = arr[i,1]
        min_max.append((menor,mayor))

    for i in range(len(min_max)):
        print(min_max[i], colums[i])

    cl = []
    descartados = []
    by_delete = []
    for i in range(len(min_max)):
        temp= []
        temp= colums[i]
        if i in descartados: continue
        for j in range(len(min_max)):

            if i==j: continue
            if min_max[i][0]-10 <= min_max[j][0] and min_max[i][1]+10 >= min_max[j][1]:
                temp += colums[j]
                descartados.append(j)
                if i > j:
                    by_delete.append(colums[j])
        cl.append(temp)

    colums = cl
    print("estos son a eliminar", by_delete)
    for k in by_delete:
        if k in colums:
            colums.remove(k)

    print()
    zises = []
    for element in colums:
        print(element)
        zises.append(len(element))
    
    mean = st.mean(zises)
    print("la media de los len ", mean)
    by_delete = []
    for i in range(len(colums)):
        percent20 = (mean/100)*20
        #print("percent20: ", percent20)
        if len(colums[i]) < percent20:
            by_delete.append(colums[i])

    for element in colums:
        print(element)
        print

    print("elementos a eliminar: ", by_delete)
    for k in by_delete:
        if k in colums:
            colums.remove(k)

    for element in colums:
        print(element)

    min_max_h= []
    for element in colums:   
        menor = arr[element[0],0]
        mayor = arr[element[0],1]
        for i in element:
            if arr[i,0] < menor:
                menor = arr[i,0]
            
            if arr[i,1] > mayor:
                mayor = arr[i,1]
        min_max_h.append((menor,mayor))

    min_max_h_unOrder = min_max_h.copy()
    min_max_h.sort()

    cl = []
    for i in np.arange(len(colums)):
        for j in np.arange(len(colums)):
            if min_max_h_unOrder[j] == min_max_h[i]:
                cl.append(colums[j])

    colums = cl

    print(min_max_h)
    
    return min_max_h,colums

El metodo build_df toma el arreglo con las columnas separadar y utiliza el diccionario construido en extracion_colums 
para crear un objeto tipo pandas.DataFrame con la misma distribucion de los datos que estaba en la imagen

In [9]:
def build_df(data_organizer, colums):
    pd_dataOrdenizer = pd.DataFrame(data_organizer)
    temp = pd.DataFrame()
    pd_columns = pd.DataFrame()
    for i in range(len(colums)):
        temp = pd.DataFrame.copy(pd_dataOrdenizer.loc[colums[i], ["text", "line"]])
        temp = temp.drop_duplicates()
        temp = temp.sort_values("line")
        temp = pd.Series(temp["text"].tolist(), index=temp["line"].tolist())
        temp = temp.loc[~temp.index.duplicated(keep='first')]
        pd_columns = pd.concat([pd_columns, temp], axis=1, ignore_index=True)

    pd_columns = pd_columns.sort_index()
    return pd_columns

In [ ]:
def del_headEnd(df):
    
    
    return df

In [143]:
#main
#path = r'./reto/1.PNG'
path = r'.\reto\3.PNG'
image = cv2.imread(path)
gray = get_grayscale(image)
th = thresholding(gray)
openingg = opening(gray)
cannyy = canny(gray)
img = image
hImg, wImg, _ = img.shape

custom_config = r'-c tessedit_char_blacklist=$¢§_{} --oem 3 --psm 4' 
data = pytesseract.image_to_data(img, lang= 'eng+spa', config=custom_config, output_type= 'dict')


In [157]:
pd.DataFrame(data).head(20)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
0,1,1,0,0,0,0,0,0,865,826,-1,
1,2,1,1,0,0,0,267,20,332,81,-1,
2,3,1,1,1,0,0,267,20,332,81,-1,
3,4,1,1,1,1,0,364,20,123,15,-1,
4,5,1,1,1,1,1,364,20,76,15,88.391655,Openatlas
5,5,1,1,1,1,2,445,21,42,11,87.629128,S.A.S.
6,4,1,1,1,2,0,355,43,142,11,-1,
7,5,1,1,1,2,1,355,43,28,11,90.626038,NIT
8,5,1,1,1,2,2,387,43,110,11,88.576340,900.465.720-5
9,4,1,1,1,3,0,267,64,332,15,-1,


In [158]:
indice_words = []
for i in range(len(data['text'])):
    if data['text'][i] != '' and data['text'][i] != ' ' and float(data['conf'][i]) >= 50 and data["text"][i] != "-" and data["text"][i] != "--" and data["text"][i] != "s" and data["text"] != "S":
        indice_words.append(i)

#print(len(indice_words))
#print(indice_words)
intervalo = calcular_espacios(data, indice_words)
print(intervalo)
coordenadas,data_organizer = coordenadas_extraccion(data,indice_words, intervalo)
df_data_organizer = pd.DataFrame(data_organizer)

(-5, 15)


In [159]:
df_data_organizer.head(10)

,text,line,index,coordinates
0,Openatlas S.A.S.,0,"[4, 5]","(364, 487)"
1,NIT 900.465.720-5,1,"[7, 8]","(355, 497)"
2,Estado de Situación,2,"[10, 11, 12]","(267, 414)"
3,Financiera Comparativo,2,"[13, 14]","(433, 599)"
4,(Expresados en Pesos Colombianos),3,"[16, 17, 18, 19]","(315, 554)"
5,31 de diciembre de,4,"[67, 68, 69, 70]","(473, 615)"
6,31 de diciembre de,4,"[71, 72, 73, 74]","(669, 811)"
7,2020,5,[76],"(528, 564)"
8,2019,5,[77],"(722, 758)"
9,NOTAS,6,[79],"(407, 459)"


In [153]:
wImg/2

432.5

In [156]:

df_data_organizer.loc[:,"coordinates"].apply(lambda x: (x[0]+x[1])/2).head(20)

0     425.5
1     426.0
2     340.5
3     516.0
4     434.5
5     544.0
6     740.0
7     546.0
8     740.0
9     433.0
10     69.0
11     94.5
12    149.0
13    433.0
14    774.5
15    197.5
16    432.5
17    578.5
18    774.0
19    142.0
Name: coordinates, dtype: float64

In [149]:


#print("numero de posiciones: " + str(len(coordenadas)))
min_max_h,colums = extracion_colums(df_data_organizer)

#Dibujado de los rectangulos en la imagen
for p in min_max_h:
    cv2.rectangle(img, (p[0], hImg-5),(p[1], 5), (random.randint(0,255), random.randint(0,255), random.randint(0,255)), 1)

for p in coordenadas:
    cv2.rectangle(img, (p[0], p[1]),(p[2], p[3]), (50, 50, 255), 1)

cv2.imwrite('salida.png',img)

df = pd.DataFrame(build_df(data_organizer, colums))
df.to_excel("data.xlsx", index=False)

"""
img = cv2.resize(img, (600, 700))
cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

el tamaño de cl es:  14
(315, 554) [0, 1, 4]
(267, 414) [2]
(433, 622) [3, 5, 17, 24, 42, 62, 74, 90, 29, 36, 39, 47, 51, 55, 59, 79, 82, 85, 88]
(669, 816) [6, 25, 40, 43, 63, 75, 91, 14, 18, 30, 34, 48, 52, 56, 60, 80, 83, 86, 89]
(528, 564) [7]
(722, 758) [8]
(407, 459) [9]
(34, 366) [10, 11, 41, 44, 73, 76, 81, 92, 23, 26, 27, 31, 35, 38, 45, 53, 57, 61, 64, 67, 70, 78, 12, 15, 19, 49, 84, 87]
(428, 437) [13, 16, 20, 28, 32, 46, 50]
(612, 622) [21, 33, 65, 68, 71]
(807, 816) [22, 37, 66, 69, 72]
(424, 442) [54, 58, 77]
(129, 137) [93]
(144, 227) [94]
estos son a eliminar []

[0, 1, 4, 7, 9, 13, 16, 20, 28, 32, 46, 50, 54, 58, 77]
[2]
[3, 5, 17, 24, 42, 62, 74, 90, 29, 36, 39, 47, 51, 55, 59, 79, 82, 85, 88, 7, 13, 16, 20, 28, 32, 46, 50, 21, 33, 65, 68, 71, 54, 58, 77]
[6, 25, 40, 43, 63, 75, 91, 14, 18, 30, 34, 48, 52, 56, 60, 80, 83, 86, 89, 8, 22, 37, 66, 69, 72]
[10, 11, 41, 44, 73, 76, 81, 92, 23, 26, 27, 31, 35, 38, 45, 53, 57, 61, 64, 67, 70, 78, 12, 15, 19, 49, 84, 87, 93, 

"\nimg = cv2.resize(img, (600, 700))\ncv2.imshow('Result', img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [150]:
df

,0,1,2,3
0,NaN,Openatlas S.A.S.,NaN,NaN
1,NaN,NIT 900.465.720-5,NaN,NaN
2,NaN,NaN,Financiera Comparativo,NaN
3,NaN,(Expresados en Pesos Colombianos),NaN,NaN
4,NaN,NaN,31 de diciembre de,31 de diciembre de
5,NaN,2020,2020,2019
6,NaN,NOTAS,NaN,NaN
7,ACTIVOS,NaN,NaN,NaN
8,Activo corriente,NaN,NaN,NaN
9,Efectivo y equivalentes de efectivo,3,3,"244,898,319"
